# Google Colab Notebook
Use this notebook to run Woogle Maps in Google Colab. Simply run all cells sequentially. It will take a few minutes to set up.

## Step 1: Get the correct python version
By default, Google Colab uses Python 3.10, but we need Python 3.12 for Woogle Maps. The following cell install the correct Python version.

In [1]:
!apt-get update
!apt-get install python3.12 python3.12-distutils

# Change 'python' to point at the newly installed version
!sudo update-alternatives --install /usr/bin/python3 python3 /usr/bin/python3.10 1
!sudo update-alternatives --install /usr/bin/python3 python3 /usr/bin/python3.12 2

# Check that it prints "Python 3.12.2"
!python --version

## Step 2: Clone the repository from GitHub

Download the source code and checkout a branch that has been tested to work in this notebook. Ignore any warnings that mention `detached HEAD`.

In [2]:
!git clone https://github.com/TeamEpochGithub/woogle-maps

## Step 3: Install all dependencies

Since dependencies are managed with Poetry, we need to install pip and Poetry first before we can install any dependencies.

In [3]:
# Install pip using get-pip
!wget https://bootstrap.pypa.io/get-pip.py
!python get-pip.py
!python3.12 -m pip install --upgrade pip --user
!pip -V

In [4]:
# Use pip to install Poetry
!python -m pip install poetry

In [5]:
# Install python devtools, necessary for building some dependencies
!apt-get install python3.12-dev

In [6]:
# Use poetry to install or build all dependencies
!cd woogle-maps; python -m poetry -q install && echo 'Finished!' || 'Failed!'

In [7]:
# Sanity check
!echo Missing dependencies:
!cd woogle-maps; python3.12 -m poetry install --dry-run | grep "\- Installing" | grep -v "Already installed" || echo "No missing dependencies found, sucess!"

In [8]:
# Last things to set up
!cd woogle-maps; poetry run python -c "import nltk; nltk.download('punkt')"

## Step 4: Setup a browser window to access the app

Since localhost is not available on Google Colab, we need ngrok to access the application instead. At the end of this step, you will find the link you need.

In [9]:
# Install ngrok, used to open a website server from the notebook
!curl -s https://ngrok-agent.s3.amazonaws.com/ngrok.asc \
	| sudo tee /etc/apt/trusted.gpg.d/ngrok.asc >/dev/null \
	&& echo "deb https://ngrok-agent.s3.amazonaws.com buster main" \
	| sudo tee /etc/apt/sources.list.d/ngrok.list \
	&& sudo apt update \
	&& sudo apt install ngrok

In [10]:
# Sign in (using throwaway ngrok account made for this purpose)
!ngrok config add-authtoken 2fe2QHsdRZYuNSJjKWEu0oIME8M_4PPSbvpQceV7d3RLepLea; sleep 3

In [11]:
# Run ngrok to tunnel Dash app port 8050 to the outside world.
# This command runs in the background.
get_ipython().system_raw('ngrok http 8060 &')
!sleep 3

In [12]:
# Check if its running, should print at least one line
!sudo lsof -PiTCP -sTCP:LISTEN | grep ngrok && echo "Sucess!" || echo "Failed! nothing running. Run the cell above again, then the one below to get the link"

In [13]:
# Get the public URL where you can access the Dash app. Copy this URL.
!curl -s http://localhost:4040/api/tunnels | python -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

## Step 5: Start the app, then open link above

Run this cell untill it says 'Dash is running on ...'.
Don't use the link starting with 127.0.0.1.
Instead, use the link above, ending with 'ngrok-free.app'. Click 'Visit Site'.

In [ ]:
!cd woogle-maps; poetry run python dashapp/app.py